In [32]:
import sounddevice as sd
import numpy as np
import io
import soundfile as sf
from pydub import AudioSegment
# Constants
RATE = 16000
CHANNELS = 1
def find_element_with_symbol(string_list, symbol):
    for i, string in enumerate(string_list):
        if string.startswith(symbol):
            return i
    return None
def get_default_output_device_name():
    devices = sd.query_devices()
    devices_string = str(devices)
    devices_list = devices_string.split('\n')
    row_number = find_element_with_symbol(devices_list, '<')
    device_name=devices[int(row_number)]['name']
    if device_name:
        return device_name
    else:
        return None
        
def record_system_audio(duration):
    default_output_device_name = get_default_output_device_name()
    device_info = sd.query_devices()
    # Find the loopback audio device
    system_audio_device = None
    for device in device_info:
        if device['name'] == default_output_device_name:
            system_audio_device = device
            break
    if system_audio_device:
        #channels = system_audio_device['max_output_channels']
        channels=CHANNELS
        print("channels",channels)
        #fs = system_audio_device['default_samplerate']
        fs=RATE
        print("samplerate",fs)
        frames = int(duration * fs)
        audio_data = sd.rec(frames=frames, samplerate=fs, channels=channels, dtype='float32')
        sd.wait()
        return np.squeeze(audio_data)
    else:
        print("System audio device not found.")
        return None

In [33]:
# Example usage
duration = 5  # Recording duration in seconds
audio_data = record_system_audio(duration)
if audio_data is not None:
    # Process or save the recorded audio data
    print(f"Recorded system audio with shape: {audio_data.shape}")

channels 1
samplerate 16000
Recorded system audio with shape: (80000,)


In [34]:
def save_to_byte_stream(audio_data):
    with io.BytesIO() as byte_stream:
        with sf.SoundFile(byte_stream, 'w', samplerate=RATE, channels=CHANNELS, format='WAV', subtype='PCM_16') as wav_file:
            wav_file.write(audio_data)
        byte_stream.seek(0)
        audio_segment = AudioSegment.from_file(byte_stream, format="wav")
    return audio_segment

In [35]:
def play_audio(audio_segment):
    print("Playing audio...")
    audio_data = np.array(audio_segment.get_array_of_samples(), dtype=np.int16)
    sd.play(audio_data, samplerate=RATE)
    sd.wait()

In [36]:
audio_segment = save_to_byte_stream(audio_data)
play_audio(audio_segment)

Playing audio...


# Updated

In [37]:
import sounddevice as sd
import numpy as np
import io
import soundfile as sf
from pydub import AudioSegment
# Constants
RATE = 16000
CHANNELS = 1
def find_element_with_symbol(string_list, symbol):
    for i, string in enumerate(string_list):
        if string.startswith(symbol):
            return i
    return None
def get_default_output_device_name():
    devices = sd.query_devices()
    devices_string = str(devices)
    devices_list = devices_string.split('\n')
    row_number = find_element_with_symbol(devices_list, '<')
    device_name=devices[int(row_number)]['name']
    if device_name:
        return device_name
    else:
        return None

def get_default_input_device_name():
    devices = sd.query_devices()
    devices_string = str(devices)
    devices_list = devices_string.split('\n')
    row_number = find_element_with_symbol(devices_list, '>')
    device_name=devices[int(row_number)]['name']
    if device_name:
        return device_name
    else:
        return None


def record_system_audio(duration):
    default_output_device_name = get_default_output_device_name()
    device_info = sd.query_devices()
    # Find the loopback audio device
    system_audio_device = None
    for device in device_info:
        if device['name'] == default_output_device_name:
            system_audio_device = device
            break
    if system_audio_device:
        #channels = system_audio_device['max_output_channels']
        #fs = system_audio_device['default_samplerate']
        channels=CHANNELS
        fs=RATE
        frames = int(duration * fs)

        audio_data = sd.playrec(np.zeros((frames, channels)), samplerate=fs, channels=channels, dtype='float32')
        sd.wait()
        return np.squeeze(audio_data)
    else:
        print("System audio device not found.")
        return None

In [41]:
# Example usage
duration = 5  # Recording duration in seconds
audio_data = record_system_audio(duration)
if audio_data is not None:
    # Process or save the recorded audio data
    print(f"Recorded system audio with shape: {audio_data.shape}")

Recorded system audio with shape: (80000,)


In [42]:
audio_segment = save_to_byte_stream(audio_data)
play_audio(audio_segment)

Playing audio...
